In [1]:
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.applications.efficientnet import EfficientNetB7
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import pandas as pd
import os
import numpy as np
import wandb
from wandb.keras import WandbCallback
from utils.ModelWrapper import ModelWrapper

In [344]:
config_defaults = {
    'epochs': 3,
    'batch_size': 32,
    'learning_rate': 0.0001,
    'dropout': 0.5,
    'regularization': 0.0001,
}
wandb.init(config=config_defaults, project="deepfake-efficientnet", entity="dat550")
config = wandb.config

accuracy,▅█▁
epoch,▁▅█
loss,█▃▁
val_accuracy,▁▁▁
val_loss,▁▄█
accuracy,0.41667
best_epoch,0
best_val_loss,1.38148
epoch,2
loss,1.63466
val_accuracy,0.5


In [345]:
batch_size = config.batch_size
img_size = 128
data_dir = "./data/tester"

model_wrapper = ModelWrapper(data_dir, img_size, config.batch_size)

In [350]:
model_file = f'f{wandb.run.name}_model.h5'

efficient_net = EfficientNetB7(
    weights = 'imagenet',
    input_shape = (img_size, img_size, 3),
    include_top = False,
    pooling = 'max',
    drop_connect_rate=0.5
)

model_layers = [
    efficient_net,
    layers.Dense(units = 512, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(config.regularization), bias_regularizer=tf.keras.regularizers.L2(config.regularization)),
    layers.Dropout(config.dropout),
    layers.Dense(units = 128, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(config.regularization), bias_regularizer=tf.keras.regularizers.L2(config.regularization)),
    layers.Dense(units = 1, activation = 'sigmoid'),
]
model_wrapper.create_model(model_file, model_layers, config)
model_wrapper.model.compile(optimizer = tf.keras.optimizers.Adam(lr=config.learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model_wrapper.model.summary()

In [354]:
# Train network
history = model_wrapper.fit()
print(history.history)

Epoch 1/3


/var/folders/sn/w2phyck94cjgjk_b1_t74h5w0000gn/T/ipykernel_49690/4008429657.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


1/1 [==============================] - ETA: 0s - loss: 0.9066 - accuracy: 0.5833
Epoch 1: val_loss improved from inf to 1.22427, saving model to fglorious-hill-8_model.h5
1/1 [==============================] - 42s 42s/step - loss: 0.9066 - accuracy: 0.5833 - val_loss: 1.2243 - val_accuracy: 0.5000 - _timestamp: 1650310957.0000 - _runtime: 54.0000
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 1.6623 - accuracy: 0.4167
Epoch 2: val_loss improved from 1.22427 to 0.80566, saving model to fglorious-hill-8_model.h5
1/1 [==============================] - 16s 16s/step - loss: 1.6623 - accuracy: 0.4167 - val_loss: 0.8057 - val_accuracy: 0.5000 - _timestamp: 1650310972.0000 - _runtime: 69.0000
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 1.1692 - accuracy: 0.5000
Epoch 3: val_loss did not improve from 0.80566
1/1 [==============================] - 4s 4s/step - loss: 1.1692 - accuracy: 0.5000 - val_loss: 1.4603 - val_accuracy: 0.5000 - _timestamp: 1650310980

In [356]:
# load the saved model that is considered the best
best_model = load_model(model_file)
model_wrapper.evaluate_model(best_model)

12/12 [==============================] - 6s 116ms/step


,Filename,Prediction
0,REAL/abqwwspghj1.jpg,0.442409
1,REAL/abqwwspghj2.jpg,0.442409
2,REAL/abqwwspghj3.jpg,0.442415
3,REAL/abqwwspghj4.jpg,0.442492
4,REAL/abqwwspghj5.jpg,0.442335
5,REAL/abqwwspghj6.jpg,0.442469
6,FAKE/abqwwspghj1.jpg,0.442409
7,FAKE/abqwwspghj2.jpg,0.442409
8,FAKE/abqwwspghj3.jpg,0.442415
9,FAKE/abqwwspghj4.jpg,0.442492


In [ ]:
model_wrapper.export_to_png()